# Tool-Augmented AgnoAgents Demo

This notebook demonstrates how to use the tool-augmented agents in the SentientResearchAgent framework.

In [ ]:
# Setup
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv()

import os
print("API Keys Available:")
print(f"- OpenAI: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")
print(f"- Anthropic: {'✅' if os.getenv('ANTHROPIC_API_KEY') else '❌'}")
print(f"- OpenRouter: {'✅' if os.getenv('OPENROUTER_API_KEY') else '❌'}")

## 1. Direct AgnoAgent Usage with Tools

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.python import PythonTools
from agno.tools.reasoning import ReasoningTools

# Import our custom web_search tool
from src.sentientresearchagent.hierarchical_agent_framework.tools.web_search_tool import web_search, clean_tools

In [ ]:
# Create a simple agent with tools
model = OpenAIChat(id="gpt-4o-mini", temperature=0.2)

# Prepare tools
tools = [
    PythonTools(base_dir=Path("tmp/python"), run_code=True),
    ReasoningTools(),
    web_search
]

# Clean tools
tools = clean_tools(tools)

# Create agent
agent = Agent(
    model=model,
    tools=tools,
    system_message="""You are a helpful research assistant with access to:
- PythonTools: Execute Python code for calculations and analysis
- ReasoningTools: Use structured reasoning
- web_search: Search for current information

Use these tools to provide accurate, well-researched answers.""",
    show_tool_calls=True,
    markdown=True
)

In [ ]:
# Test the agent
response = agent.run("Calculate the compound interest on $10,000 at 5% annual rate for 10 years, compounded monthly")
print(response.content)

In [ ]:
# Test with web search
response = agent.run("What is the current population of Tokyo?")
print(response.content)

## 2. Using Tool-Augmented Agents in the Framework

In [ ]:
# This would be used within the SentientResearchAgent framework
# The agents are configured in agents.yaml with tools

# Example configuration:
config_example = """
- name: "BasicReasoningExecutor"
  type: "executor"
  adapter_class: "ExecutorAdapter"
  description: "Performs Analysis and Synthesis of information"
  model:
    provider: "openai"
    model_id: "gpt-4o-mini"  # OpenAI models work well with tools
  agno_params:
    reasoning: true
  prompt_source: "prompts.executor_prompts.REASONING_EXECUTOR_SYSTEM_MESSAGE"
  tools: ["PythonTools", "ReasoningTools"]  # Tools configured here!
  registration:
    action_keys:
      - action_verb: "execute"
        task_type: "THINK"
    named_keys: ["BasicReasoningExecutor"]
  enabled: true
"""

print("Agent Configuration Example:")
print(config_example)

## 3. Creating Custom Tool Functions

In [ ]:
# You can create custom tool functions
def analyze_sentiment(text: str) -> str:
    """Analyze the sentiment of the given text."""
    # Simple mock implementation
    positive_words = ['good', 'great', 'excellent', 'happy', 'wonderful']
    negative_words = ['bad', 'terrible', 'awful', 'sad', 'horrible']
    
    text_lower = text.lower()
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    if positive_count > negative_count:
        return "Positive sentiment detected"
    elif negative_count > positive_count:
        return "Negative sentiment detected"
    else:
        return "Neutral sentiment detected"

# Add to agent
tools_with_custom = tools + [analyze_sentiment]
tools_with_custom = clean_tools(tools_with_custom)

agent_with_sentiment = Agent(
    model=model,
    tools=tools_with_custom,
    system_message="You are an assistant with sentiment analysis capabilities.",
    show_tool_calls=True
)

In [ ]:
# Test sentiment analysis
response = agent_with_sentiment.run("Analyze the sentiment of: 'This is a wonderful day!'")
print(response.content)

## Key Takeaways

1. **Tool Integration is Simple**: Just add tools to the `tools` list in agent configuration
2. **Model Compatibility Matters**: OpenAI models work best with tools, some models may have issues
3. **Custom Tools**: You can create any Python function as a tool
4. **Clean Tools**: Always use `clean_tools()` to remove problematic attributes
5. **Configuration**: In `agents.yaml`, add tools at the top level, not in `agno_params`